# Problem 3.4 END

Now that our frog knows how to find and grab the key, it should use it to open the door. There are two cases it needs to think about: first, what to do when it has the key, and second, what to do when it doesn't.

When the frog *has* the key, it will probably want to go looking for the door, and try to open it.

When the frog *does not have the key*, it will probably want to ignore the door, lest it be distracted from the key search.

We will teach it how to behave in two different ways when it sees the door.

### First, Install the Lab Libraries

In [ ]:
%pip install python_actr git+https://github.com/eilene-ftf/pondworld.git

  Cloning https://github.com/eilene-ftf/pondworld.git to /tmp/pip-req-build-c_py_on9
  Running command git clone --filter=blob:none --quiet https://github.com/eilene-ftf/pondworld.git /tmp/pip-req-build-c_py_on9
  Resolved https://github.com/eilene-ftf/pondworld.git to commit 7aa712ff4d7ad0034d2e95090831542552845893
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for python_actr: filename=python_actr-1.9.2-py3-none-any.whl size=

### Second, Import the Necessary Libraries

In [ ]:
import gymnasium as gym
from minigrid.minigrid_env import MiniGridEnv

import pondworld
from pondworld import FrogControl, EnvState

from python_actr import Model, ACTR, Buffer, Memory, log_everything

from IPython.display import clear_output

import time

### The Pond

**Don't forget to run this bit!**

In [ ]:
class Pond(Model):
    pass

### The Frog

Now that you're comfortable with the mental world of the frog, we're going to try to tell the frog what to do about getting into that pantry. The door is locked, and our frog's left a key somewhere around the house. As it cannot open the door without a key, the first thing to do is get the key. Once it has the key, the next job is opening the door, so it can get in and eat the flies. Try running the code first to see what's happening in the frog's world, and then go look at your task.

In [ ]:
# create our frog's brain
class FrogMind(ACTR):
    # give our frog a buffer that holds information
    # let's call this buffer "vision". it represents what the frog is seeing.
    vision=Buffer()

    # here we're going to set what the frog is seeing right at the moment
    # initially we don't know what the frog is looking at
    vision.set('unknown')

    # "possibilities" has been replaced with a set of slot names, corresponding
    # to the three items the frog might track in its world, and a set of values,
    # corresponding to what states those items might be in
    slots = ["fly", "key", "door"]
    values = ["nowhere", "here", "ahead", "left", "right", "held"]

    # also, we're keeping track of all the things the frog can do
    # you never know when you might need them
    actions = ["forward", "left", "right", "interact", "pickup"]

    ######
    # THE BASICS

    # look to see if there's any flies
    def look(vision='unknown'):
        self.wait(0.3) # adds a rest between steps so we can see what's going on
        self.parent.clear() # makes sure only the most recent output is shown
        observation = self.parent.frog_body.look()  # now, when it looks,
                                                    # the frog will get a
                                                    # collection of several
                                                    # items

        # This creates a list of 'slot:value' pairs, using what's
        # contained in observation
        new_memory = []
        for slot in slots:
            val_index = observation[slot]   # gets a number indicating the state
                                            # of the object called 'slot'
            value = values[val_index]       # then looks it up
            new_memory.append(f'{slot}:{value}') # and adds it to

        updated = ' '.join(new_memory)  # make the contents of new_memory into a
                                        # string ACT-R can understand
        print('what the frog sees: \n  ' + updated)
        vision.set(updated)

    # if you see a fly, eat a fly
    def eat_fly(vision='fly:here'):
        print("The frog spits out its tongue and snaps up the fly.")
        result = self.parent.frog_body.move("interact")
        if result.terminated:
          vision.set('done') # the frog has satisfied its hunger
        else:
          vision.set('unknown')

    ######
    # EXPLORATION

    # if you don't see a fly trying looking left
    def explore_left(vision="fly:nowhere key:nowhere"):
        print("The frog explores left.")
        self.parent.frog_body.move("left")
        vision.set('unknown')

    # if you don't see a fly try hopping forward
    def explore_forward(vision="fly:nowhere key:nowhere"):
        print("The frog explores forward.")
        self.parent.frog_body.move("forward")
        vision.set('unknown')

    # PUT TWO MORE PRODUCTIONS HERE

    ######
    # CHASE THE FLY

    # if fly is to the left, turn left
    def turn_left_fly(vision="fly:left"):
        print("The frog turns left.")
        self.parent.frog_body.move("left")
        vision.set('unknown')

    # if fly is to the right, turn right
    def turn_right_fly(vision="fly:right"):
        print("The frog turns right.")
        self.parent.frog_body.move("right")
        vision.set('unknown')

    # if you see something in the distance trying hopping forward
    def hop_forward_fly(vision="fly:ahead"):
        print("The frog hops forward.")
        self.parent.frog_body.move("forward")
        vision.set('unknown')

    ######
    # GRAB A KEY

    # if you see a key, pick it up
    def pickup_key(vision='key:here'):
        print('The frog picks up the key to the pantry.')
        self.parent.frog_body.move('pickup')
        vision.set('unknown')

    # if fly is to the left, turn left
    def turn_left_key(vision="key:left"):
        print("The frog turns left.")
        self.parent.frog_body.move("left")
        vision.set('unknown')

    # if fly is to the right, turn right
    def turn_right_key(vision="key:right"):
        print("The frog turns right.")
        self.parent.frog_body.move("right")
        vision.set('unknown')

    # if you see something in the distance trying hopping forward
    def hop_forward_key(vision="key:ahead"):
        print("The frog hops forward.")
        self.parent.frog_body.move("forward")
        vision.set('unknown')

    ######
    # GO TO DOOR - PUT FOUR PRODUCTIONS HERE

    ######
    # END STATE

    # when all the flies are eaten, it's time to stop and rest
    def end(vision='done'):
        print("The frog is full.")
        self.stop()

### Your Task

What happens now may have surprised you! As soon as the frog picks up the key, it gets confused and gives up. Clearly, this isn't the outcome it wanted, so we need to teach it what to do when it is holding the key.

**Your job is to write six more productions**.

**Two** of them will describe how the frog behaves when it is holding the key, but does not see a door. **They should mimic the existing exploration behaviour**. We want the frog to look around for the door.

**Four** more productions will describe what happens when the frog is holding the key and sees the door. **They should be based on what the frog does when it sees a fly**.

One of the four productions should **open the door**. To open the door use:
```python
        self.parent.frog_body.move('interact')
```

**All of these productions must be designed so that they only fire when the frog is holding the key**. We want to make sure the frog isn't distracted by trying to go into the pantry when it isn't able to. **How do you need to change their conditions in order to make that happen**?

You may notice the frog opening the door and then immediately turning around and walking away. This behaviour can be mitigated by having the frog take an extra hop forward as part of the same production rule that opens the door.

**Your second job is to answer these two short-answer questions**. (Write no more than two or three sentences).

### Questions

#### Question 1

Based on your readings and your understanding so far, what could be done to the frog to make it a better ACT-R model? Are there details about the philosophy of ACT-R or the implementation of the frog that are lacking?  Write your answer below the break by double clicking this box.


---

#### Question 2

After completing each exercise, the next notebook contained the solutions to the one preceding it. Did you notice any mistakes in your work? Something you still don't understand? What were they?

---

### Run Your Model

Make sure everything works by running the code blocks below.

In [ ]:
# Sets up the frog's virtual world
env = gym.make("house-v0", tile_size=32, render_mode='rgb_array')
# Gives the frog a body in it
frog_body = FrogControl(env, textmode=True, emojis=True)
# Gets things going!
frog_body.start()

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.gen_obs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.gen_obs` for environment variables or `env.get_wrapper_attr('gen_obs')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.mission to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.mission` for environment variables or `env.get_wrapper_attr('mission')` that will search the reminding wrappers.
  logger.warn(


In [ ]:
kermit = FrogMind()                       # name the agent
kermit.wait = time.sleep
paradise_swamp = Pond()                   # name the environment
paradise_swamp.clear = lambda: clear_output(wait=True)
paradise_swamp.frog_body = frog_body      # put the frog's body in the environment
paradise_swamp.agent = kermit             # put the frog's brain in the environment
paradise_swamp.run()                      # Annnnd action!

🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱    
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱    viewfinder:
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱    viewfinder:
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱    🪰⬜⬛⬛🗝️🟨⬛
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱    live frog reaction:
🧱⬛⬛⬛🟨🟨🟨🟨🧱🟨🟨⬛⬛⬛⬛🧱    🧱🟨🟨🟨🟨🟨🟨
🧱⬛⬛⬛🟨🟨🟨🟨🧱🟨🟨⬛⬛⬛⬛🧱    🧱🟨🟨🟨🟨🟨🟨
🧱⬛⬛⬛🟨🟨🟨🟨🧱🟨🟨⬛⬛⬛⬛🧱    🧱🚪🧱🧱🧱🧱🧱
🧱⬛⬛⬛🐸🪰🟨🟨🧱🟨🟨⬛⬛⬛⬛🧱    🧱🟨🟨🟨🟨🟨🟨
🧱⬛⬛⬛🟨🟨🟨🟨🧱🟨🟨⬛⬛⬛⬛🧱    🧱🟨🟨🟨🟨🟨🟨
🧱⬛⬛⬛🟨🟨🟨🟨🚪🟨🟨⬛⬛⬛⬛🧱    🧱🟨🟨🪰🟨🟨🟨
🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱    🧱🟨🟨🐸🟨🟨🟨
eat all the flies
frog compass: ➡️
what the frog sees: 
  fly:here key:held door:ahead
The frog spits out its tongue and snaps up the fly.
The frog is full.
